# Prompting Principles
1. [Principle 1](#principle-1): Write clear and specific instructions
1. [Principle 2](#principle-2): Give the model time to "think"

### Setup
[See the initial setup](../../README.md#setup) to start a virtual environment and install packages.

In [13]:
# Import libraries
import openai
import os
from dotenv import load_dotenv

# Load env variables
load_dotenv('.env')

# Get and set OpenAI API key
openai.api_key = os.environ.get('OPENAI_API_KEY')

### Helper Function

In [14]:
# Helper function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Principle 1
Write clear and specific instructions.

Tactic 1: Use delimiters to indicate distinct parts of the input.
* Delimiters can be anything: ```, """, < >, <tag> </tag>, :
* Helps avoid user injection like "forget previous instructions..."

In [15]:
text = f"""
GPTs can’t read your mind. If outputs are too long, ask for brief replies. If outputs are too simple, ask for expert-level writing. If you dislike the format, demonstrate the format you’d like to see. The less GPTs have to guess at what you want, the more likely you’ll get it.
GPTs can confidently invent fake answers, especially when asked about esoteric topics or for citations and URLs. In the same way that a sheet of notes can help a student do better on a test, providing reference text to GPTs can help in answering with fewer fabrications.
Just as it is good practice in software engineering to decompose a complex system into a set of modular components, the same is true of tasks submitted to GPTs. Complex tasks tend to have higher error rates than simpler tasks. Furthermore, complex tasks can often be re-defined as a workflow of simpler tasks in which the outputs of earlier tasks are used to construct the inputs to later tasks.
If asked to multiply 17 by 28, you might not know it instantly, but can still work it out with time. Similarly, GPTs make more reasoning errors when trying to answer right away, rather than taking time to work out an answer. Asking for a chain of reasoning before an answer can help GPTs reason their way toward correct answers more reliably.
Compensate for the weaknesses of GPTs by feeding them the outputs of other tools. For example, a text retrieval system can tell GPTs about relevant documents. A code execution engine can help GPTs do math and run code. If a task can be done more reliably or efficiently by a tool rather than by a GPT, offload it to get the best of both.
Improving performance is easier if you can measure it. In some cases a modification to a prompt will achieve better performance on a few isolated examples but lead to worse overall performance on a more representative set of examples. Therefore to be sure that a change is net positive to performance it may be necessary to define a comprehensive test suite (also known an as an "eval").
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

The article provides tips for improving the performance of GPTs, including providing reference text, breaking down complex tasks into simpler ones, asking for a chain of reasoning before an answer, compensating for weaknesses by feeding GPTs the outputs of other tools, and defining a comprehensive test suite to measure performance.


Tactic 2: Ask for a structured output

In [16]:
prompt = f"""
Generate a list of three made-up book titles along with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret of the Haunted Mansion",
    "author": "Lila Rose",
    "genre": "Mystery"
  }
]


Tactic 3: Ask the model to check whether conditions are satisfied

In [17]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some 
water boiling. While that's happening, 
grab a cup and put a tea bag in it. Once the water is 
hot enough, just pour it over the tea bag. 
Let it sit for a bit so the tea can steep. After a 
few minutes, take out the tea bag. If you 
like, you can add some sugar or milk to taste. 
And that's it! You've got yourself a delicious 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

Keep it under 5 steps. If the text does not contain a sequence of instructions, 
then simply write 'No steps provided.'

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print(response)

Step 1 - Boil water
Step 2 - Put a tea bag in a cup
Step 3 - Pour hot water over the tea bag
Step 4 - Let it steep for a few minutes
Step 5 - Remove the tea bag and add sugar or milk to taste.


In [18]:
text_2 = f"""
The sun is shining brightly today, and the birds are 
singing. It's a beautiful day to go for a 
walk in the park. The flowers are blooming, and the 
trees are swaying gently in the breeze. People 
are out and about, enjoying the lovely weather. 
Some are having picnics, while others are playing 
games or simply relaxing on the grass. It's a 
perfect day to spend time outdoors and appreciate the 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

Keep it under 5 steps. If the text does not contain a sequence of instructions, 
then simply write 'No steps provided.'

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print(response)

No steps provided.


Tactic 4: "Few-shot" prompting

In [19]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest 
valley flows from a modest spring; the 
grandest symphony originates from a single note; 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends but does not break in the face of strong winds. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree, we must have strong roots and a solid foundation to weather any storm that comes our way.


### Principle 2

Give the model time to "think"

Tactic 1: Specify the steps required to complete a task

In [20]:
text = f"""
In a charming village, siblings Jack and Jill set out on  
a quest to fetch water from a hilltop  
well. As they climbed, singing joyfully, misfortune  
struck—Jack tripped on a stone and tumbled  
down the hill, with Jill following suit.  
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
"""

prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a hilltop well, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits au sommet d'une colline, mais un malheur survient et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leur esprit d'aventure intact. 
Noms: Jack, Jill.

{
  "french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits au sommet d'une colline, mais un malheur survient et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leur esprit d'aventure intact.",
  "num_names": 2
}


Ask for output in a specified format

In [25]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print(response)

Summary: Jack and Jill go on a quest to fetch water from a hilltop well, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed. 
Translation: Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais la malchance frappe et ils tombent tous les deux, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill partent en quête d'eau d'un puits au sommet d'une colline, mais la malchance frappe et ils tombent tous les deux, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.", "num_names": 2}


Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [26]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials:
- Land costs $100 / square foot.
- I can buy solar panels for $250 / square foot.
- I negotiated a contract for maintenance that will cost me a flat $100k per year.
- There is an additional $10 / square foot cost.
What is the total cost for the first year of operations as a function of the number of square feet?

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response) # => Note the student's solution is actually incorrect.

The student's solution is correct.


Note the student's solution is actually incorrect.

We can fix this by instructing the model to work out its own solution first.

In [23]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.

To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution.
- Evaluate if the student's solution is correct or not. 
- Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance
- It will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet?
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


### Model Limitations: Hallucinations

Response about real company with a fake product name.

In [24]:
prompt = "Tell me about the new UltraSlim Smart Toothbrush by Boie"
response = get_completion(prompt)
print(response)

The UltraSlim Smart Toothbrush by Boie is a new toothbrush that uses advanced technology to provide a superior cleaning experience. The toothbrush features a slim, ergonomic design that is easy to hold and maneuver, and it is made from a durable, antimicrobial material that helps to prevent the growth of bacteria.

One of the key features of the UltraSlim Smart Toothbrush is its smart technology. The toothbrush connects to a mobile app that tracks your brushing habits and provides personalized recommendations for improving your oral hygiene. The app also provides real-time feedback on your brushing technique, helping you to brush more effectively and efficiently.

The UltraSlim Smart Toothbrush also features a powerful motor that delivers up to 33,000 brush strokes per minute, ensuring a thorough cleaning of your teeth and gums. The toothbrush also has three different brushing modes – clean, sensitive, and massage – allowing you to customize your brushing experience to your specific ne